In [51]:
import psycopg2
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from sqlalchemy import create_engine, text

In [4]:
# Define database connection parameters
database_connection = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'vic_db',
}


# A psycopg2 connection and cursor
conn = psycopg2.connect(user=database_connection['username'],
                        password=database_connection['password'],
                        host=database_connection['host'],
                        port=database_connection['port'],
                        database=database_connection['database'])
conn.autocommit = True
cursor = conn.cursor()

# Create a SQLAlchemy engine
engine = create_engine('postgresql://%(username)s:%(password)s@%(host)s/%(database)s' % database_connection, isolation_level="AUTOCOMMIT")
conn_alchemy = engine.connect()

In [41]:

# Define the SQL query to select data from the PostGIS table
sql = "SELECT * FROM vmtrans.tr_road_all LIMIT 10;"

# Read data from PostGIS into a GeoDataFrame
gdf = gpd.read_postgis(sql, con=engine)

# # Print the GeoDataFrame
# print(gdf)


In [9]:
# List all tables
cursor.execute("SELECT table_schema, table_name FROM information_schema.tables;")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [11]:
cursor.execute("SELECT * FROM vmtrans.tr_road_all LIMIT 10;")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [37]:
# In table tr_road_all, create a column called "road_length" which is the length of the road segment
# But not really create it, just create a new column from SELECT statement. Use meters as unit, so we have to convert the length from degree to meters
# cursor.execute("SELECT SUM(ST_Length(geom::geography)) as road_length FROM vmtrans.tr_road_all WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';")
cursor.execute(
"""
SELECT 
    ST_Length(geom::geography) as road_length,
    ST_AsText(ST_StartPoint(ST_GeometryN(geom, 1))) AS first_point,
    ST_AsText(ST_EndPoint(ST_GeometryN(geom, 1))) AS last_point
FROM vmtrans.tr_road_all 
WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';
""")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
result_df

,road_length,first_point,last_point
0,74.321053,POINT(144.7845466330001 -37.765636496999946),POINT(144.78466143100002 -37.76497362999993)
1,114.023511,POINT(145.0549611360001 -37.86946304899993),POINT(145.05515306400002 -37.86844708399997)
2,271.300323,POINT(145.12387210600002 -37.90911573999995),POINT(145.12431451100008 -37.90669674099996)
3,52.212317,POINT(145.0548732430001 -37.86992826699998),POINT(145.0549611360001 -37.86946304899993)
4,247.812288,POINT(144.78573093800003 -37.76742263799997),POINT(144.7845466330001 -37.765636496999946)
5,13.516021,POINT(145.12301246200002 -37.91167085699993),POINT(145.12310854800012 -37.911575813999946)
6,287.039748,POINT(145.12310854800012 -37.911575813999946),POINT(145.12387210600002 -37.90911573999995)
7,17.342869,POINT(145.1290169560001 -38.05468524999998),POINT(145.12883521900005 -38.05474658899993)
8,301.855147,POINT(145.05515306400002 -37.86844708399997),POINT(145.0556611290001 -37.86575751099997)
9,164.729226,POINT(144.7879104740001 -37.76769546799994),POINT(144.7860614210001 -37.76747599199996)


In [59]:
# sql = """
# SELECT ezi_road_name, ST_Length(geom::geography) as road_length, ST_AsText(ST_StartPoint(ST_GeometryN(geom, 1))) AS first_point, ST_AsText(ST_EndPoint(ST_GeometryN(geom, 1))) AS last_point FROM vmtrans.tr_road_all WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';
# """
sql = """
SELECT
    GeometryType(geom) AS geometry_type,
    COUNT(*) AS count
FROM vmtrans.tr_road_all
GROUP BY GeometryType(geom);
"""

pd.read_sql_query(text(sql), conn_alchemy)

,geometry_type,count
0,MULTILINESTRING,1222415


In [68]:
sql = """
SELECT
    from_ufi,
    to_ufi,
    ST_X(first_point::geometry) AS first_point_x,
    ST_Y(first_point::geometry) AS first_point_y,
    ST_X(last_point::geometry) AS last_point_x,
    ST_Y(last_point::geometry) AS last_point_y
FROM vmtrans.tr_road_all
LIMIT 10;
"""
# gpd.read_postgis(text(sql), con=engine)
pd.read_sql_query(text(sql), conn_alchemy)

,from_ufi,to_ufi,first_point_x,first_point_y,last_point_x,last_point_y
0,2036059.0,2036057.0,144.632226,-36.182305,144.635463,-36.182305
1,15554142.0,16288188.0,144.982563,-37.834037,144.983084,-37.833828
2,18459581.0,18459580.0,145.239520,-38.448936,145.239522,-38.448835
3,16725603.0,16384630.0,144.332436,-37.398808,144.333167,-37.399693
4,2408990.0,16729012.0,144.352495,-38.134773,144.353832,-38.134926
5,16288145.0,16288142.0,144.974104,-37.828773,144.973387,-37.827878
6,13177738.0,2273314.0,146.056055,-37.726144,146.055975,-37.726238
7,15347009.0,15347010.0,144.729790,-36.133315,144.730937,-36.133320
8,13157990.0,13157986.0,144.340383,-36.800249,144.340994,-36.799485
9,16678757.0,16678759.0,145.150687,-37.339803,145.149248,-37.340359


In [ ]:
# Add new columns to the table
sql = """
ALTER TABLE vmtrans.tr_road_all
ADD COLUMN road_length_meters double precision,
ADD COLUMN from_point geometry(Point, 7844),
ADD COLUMN to_point geometry(Point, 7844);

UPDATE vmtrans.tr_road_all
SET road_length_meters = ST_Length(geom::geography),
    from_point = ST_StartPoint(ST_GeometryN(geom, 1)),
    to_point = ST_EndPoint(ST_GeometryN(geom, 1));
"""
cursor.execute(sql)
# 1m 30s

In [97]:
# vmtrans.tr_road_all has columns from_ufi, to_ufi, from_point, to_point
# Create a new table from vmtrans.tr_road_all, with 2 columns: ufi, point
# The new table should have 2 rows for each row in vmtrans.tr_road_all, one for from_ufi and from_point, one for to_ufi and to_point
sql = """
CREATE TABLE vmtrans.tr_points AS
SELECT from_ufi AS ufi, from_point AS geom FROM vmtrans.tr_road_all
UNION ALL
SELECT to_ufi AS ufi, to_point AS geom FROM vmtrans.tr_road_all;
"""
cursor.execute(sql)
# 17s - 30s

# Check that each ufi has only one point
sql = """
SELECT ufi, COUNT(DISTINCT geom) AS count
FROM vmtrans.tr_points
GROUP BY ufi
HAVING COUNT(DISTINCT geom) > 1;
"""
cursor.execute(sql)
result = cursor.fetchall()
assert len(result) == 0

# Remove duplicate ufi-geom pairs
sql = """
CREATE TABLE vmtrans.tr_points_clean AS
SELECT ufi, geom
FROM vmtrans.tr_points
GROUP BY ufi, geom;
"""
cursor.execute(sql)

# Rename the table from tr_points_clean to tr_points
sql = """
DROP TABLE IF EXISTS vmtrans.tr_points;
ALTER TABLE vmtrans.tr_points_clean
RENAME TO tr_points;
"""
cursor.execute(sql)

# Assert that ufi is unique
cursor.execute("SELECT COUNT(ufi), COUNT(DISTINCT ufi) FROM vmtrans.tr_points;")
result = cursor.fetchall()
assert result[0][0] == result[0][1]

In [98]:
gpd.read_postgis("SELECT * FROM vmtrans.tr_points LIMIT 10;", con=engine)

,ufi,geom
0,54637436.0,POINT (148.21369 -35.75781)
1,59918101.0,POINT (145.65433 -36.30990)
2,60425899.0,POINT (149.58132 -36.06935)
3,49754475.0,POINT (147.17442 -35.65763)
4,51705580.0,POINT (143.86532 -37.58262)
5,46722523.0,POINT (146.72891 -36.34118)
6,60416152.0,POINT (148.56595 -35.09645)
7,54757879.0,POINT (144.60838 -36.24235)
8,49692680.0,POINT (144.42042 -34.85279)
9,55247502.0,POINT (146.99632 -36.24295)
